# Psi4Numpy Demo
In this lesson we will demonstrate how to run basic calculations in psi4, calculate a PES, and fit that surface to a Lennard-Jones function.  

In [ ]:
import psi4
import numpy
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

In [ ]:
he = psi4.geometry("""
He
""")

In [ ]:
energy_he = psi4.energy('scf/cc-pVDZ', molecule=he)
print(F'Done calculating energy! The energy in Hartree is {energy_he:.5f}.')

In [ ]:
h2o = psi4.geometry("""
O
H 1 0.96
H 1 0.96 2 104.5
""")

In [ ]:
energy_h2o_optimized = psi4.optimize('scf/cc-pVDZ', molecule=h2o)
print(F'Energy of optimized H2O is {energy_h2o_optimized}.')

In [ ]:
he_dimer = psi4.geometry("""
He
--
He 1 1.0
""")
int_energy = psi4.energy('MP2/aug-cc-pVDZ', molecule=he_dimer, bsse_type='cp')
int_energy = int_energy*219474.6  # in wavenumbers 
print(F'The interaction energy in wavenumbers is {int_energy}.')

Now that we know how to run an interaction energy calculation, we can calculate a potential energy surface by calculating the interaction energy at many different internuclear separations.  Is the smart way to do this to copy and paste the code block above and change the number a bunch of times?  NO!  We should make a list of the distances we want to use for the internuclear separations and then use a for loop to run our calculations for each of those distances.  Complete the code block below to calculate the energies and append them to a new list called energies. When you have finished the calculations, make a graph of the energy (y) vs. internuclear separation (x).  

In [ ]:
he_dimer_template = """
He
--
He 1 **R**
"""

distances = [2.875, 3.0, 3.125, 3.25, 3.375, 3.5, 3.75, 4.0, 4.5, 5.0, 6.0]  # in angstoms
energies = []

for d in distances:
    mol = psi4.geometry(he_dimer_template.replace('**R**', str(d)))
    #Write your code here; insert additional cells if needed
    

Now that we have our PES calculated, we want to fit it to a Morse potential.  We will learn how to use a new function from the scipy library to do this in a moment, but first we need to define the function we want to fit.  Write a function for the Lennard-Jones function.
$$ V = 4 \varepsilon \left[ \left( \frac{\sigma}{r} \right)^{12} - \left( \frac{\sigma}{r} \right)^6 \right] $$
Your function should accept three inputs: a list of r values, $\sigma$, and $\varepsilon$.  Eventually, we will be finding the values of $\sigma$ and $\varepsilon$ that best fit our data.

In [ ]:
# Write you function here.  For consistency later, name it LJ_func



Now we will use a function called `curve_fit` from the library `scipy` to fit our data to the Lennard-Jones functional form.  You may have noticed that we imported this function at the top of the notebook.  Since `scipy` is a large library and we just want to use one particular function, we just imported the one function we plan to use.

In [ ]:
params, variance = curve_fit(LJ_func, distances, energies)
calc_epsilon=params[0]
calc_sigma = params[1]
print(F'epsilon={calc_epsilon:.3f} sigma={calc_sigma:.3f}')

Finally we want to assess the quality of our fit.  To do this, we want to use our function we wrote before and the cacluated $\varepsilion$ and $\sigma$ parameters to graph our fit curve.

In [ ]:
rpoints = numpy.linspace(2.875,7.0,100)
fit_energies = LJ_func(rpoints,calc_epsilon,calc_sigma)
plt.scatter(distances,energies)
plt.plot(rpoints,fit_energies)